In [4]:
# 구미 버스 정류장 API로 가져오기

import requests

url = 'http://apis.data.go.kr/5080000/busSttnService/getBusSttn'
params ={'serviceKey' : 'wRGALjymSzbi/GLVDdu69u7/jHfZ6d1luMJ6H3t3uCt5BETeM0wzS48xVuJ0Gv8lqct7bqmHnr6jA47/T4OvLA==',
          'pageNo' : 1, 'numOfRows' : '10000'}

response = requests.get(url, params=params)
print(response.text)

{"body":[{"sttn_idntfr":1,"sttn_std_idntfr":355000923,"sttn_ty":1,"centr_cartrk_se":0,"korean_nm":"4공단(현진아파트방면)","shrten_nm":"4공단","la":128.4295512,"lo":36.13721741,"detect_headg_value":285,"link_idntfr":3550007900,"link_la":128.4295593,"link_lo":36.1371692,"link_isesttn_sn":2,"link_isesttn_dstnc":700.0,"start_dcs_dstnc":50,"arvl_dcs_dstnc":50,"reform_no":1399,"sttn_svc_idntfr":10001,"sttn_svc_nm":"4공단"},{"sttn_idntfr":2,"sttn_std_idntfr":355000713,"sttn_ty":0,"centr_cartrk_se":0,"korean_nm":"4공단(우미린더스카이건너방면)","shrten_nm":"4공단","la":128.429585,"lo":36.136961,"detect_headg_value":85,"link_idntfr":3550050600,"link_la":128.4281314,"link_lo":36.13675458,"link_isesttn_sn":1,"link_isesttn_dstnc":40.0,"start_dcs_dstnc":50,"arvl_dcs_dstnc":50,"reform_no":1039,"sttn_svc_idntfr":10002,"sttn_svc_nm":"4공단"},{"sttn_idntfr":3,"sttn_std_idntfr":355000381,"sttn_ty":0,"centr_cartrk_se":0,"korean_nm":"4공단입구건너(인동농협옥계지점)","shrten_nm":"4공단입구","la":128.4173574,"lo":36.13781552,"detect_headg_value":188,"link

In [5]:
import json

# 받은 JSON 데이터 파싱
data = json.loads(response.text)

# 정상적으로 변환되었는지 확인
print(type(data))  # dict 형태면 정상
print(data.keys())  # "body"와 "header" 키가 있어야 정상


<class 'dict'>
dict_keys(['body', 'header'])


In [6]:
import pandas as pd
df = pd.DataFrame(data['body'])
df.head() # link_la, link_lo 은 버스가 정차하는 위치, la, lo는 버스 정류장의 위치

,sttn_idntfr,sttn_std_idntfr,sttn_ty,centr_cartrk_se,korean_nm,shrten_nm,la,lo,detect_headg_value,link_idntfr,link_la,link_lo,link_isesttn_sn,link_isesttn_dstnc,start_dcs_dstnc,arvl_dcs_dstnc,reform_no,sttn_svc_idntfr,sttn_svc_nm
0,1,355000923,1,0,4공단(현진아파트방면),4공단,128.429551,36.137217,285,3550007900,128.429559,36.137169,2,700.0,50,50,1399.0,10001,4공단
1,2,355000713,0,0,4공단(우미린더스카이건너방면),4공단,128.429585,36.136961,85,3550050600,128.428131,36.136755,1,40.0,50,50,1039.0,10002,4공단
2,3,355000381,0,0,4공단입구건너(인동농협옥계지점),4공단입구,128.417357,36.137816,188,3550008900,128.417503,36.137799,1,99.4,50,50,1278.0,10003,4공단입구건너(인동농협옥계지점)
3,4,355000383,0,0,거상빌딩,거상빌딩,128.417043,36.136046,188,3550009100,128.417155,36.136034,1,78.8,50,50,NaN,10004,거상빌딩
4,5,355000685,0,0,4공단입구(옥계방면),4공단입구,128.417766,36.137739,8,3550051500,128.417630,36.137754,1,117.0,50,50,NaN,10005,4공단입구


In [7]:
# df.to_csv('버스_정류장.csv', index=False, encoding='utf-8-sig')

In [8]:
df.shape

(2115, 19)

In [9]:
df.sttn_ty.value_counts()

sttn_ty
0    1778
1     337
Name: count, dtype: int64

In [10]:
df1 = df[['sttn_idntfr', 'korean_nm', 'link_la', 'link_lo']]
df1.head(3)

,sttn_idntfr,korean_nm,link_la,link_lo
0,1,4공단(현진아파트방면),128.429559,36.137169
1,2,4공단(우미린더스카이건너방면),128.428131,36.136755
2,3,4공단입구건너(인동농협옥계지점),128.417503,36.137799


In [57]:
import osmnx as ox
import networkx as nx

# OSM에서 도로 네트워크 데이터 가져오기 (구미시 기준)
place_name = "Gumi, South Korea"
G = ox.graph_from_place(place_name, network_type="drive")  # 차량 도로 네트워크

# 출발지와 도착지 (위도, 경도 입력)
origin_lat, origin_lon = df1[df1.sttn_idntfr == 1].link_la.values[0], df1[df1.sttn_idntfr == 1].link_lo.values[0]  # 4공단(현진아파트방면)
destination_lat, destination_lon = df1[df1.sttn_idntfr == 6].link_la.values[0], df1[df1.sttn_idntfr == 6].link_lo.values[0]  # 우미린더스카이앞

# 가장 가까운 도로 노드 찾기
orig_node = ox.distance.nearest_nodes(G, origin_lon, origin_lat)
dest_node = ox.distance.nearest_nodes(G, destination_lon, destination_lat)

# 실제 도로를 따라 최단 거리 찾기
route_length = nx.shortest_path_length(G, orig_node, dest_node, weight="length")

# 결과 출력 (미터 → km 변환)
print(f"출발지 → 도착지 실제 도로 이동 거리: {route_length / 1000:.8f} km")


출발지 → 도착지 실제 도로 이동 거리: 0.00000000 km


### ***tmap api로 탐색 거리 계산***

In [ ]:
# 출발점과 도착점 좌표
display(df1[df1.sttn_idntfr == 1].link_la.values[0], df1[df1.sttn_idntfr == 1].link_lo.values[0])  # 4공단(현진아파트방면)
display(df1[df1.sttn_idntfr == 6].link_la.values[0], df1[df1.sttn_idntfr == 6].link_lo.values[0])  # 우미린더스카이앞

np.float64(128.4295593)

np.float64(36.1371692)

np.float64(128.4328746)

np.float64(36.13784296)

In [19]:
import requests

url = "https://apis.openapi.sk.com/tmap/routes"

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "appKey": "nB9HlUzai25u6j9f7snwOauFMVA1DZbA2NGSgNid"
}

data = {
    "startX": 128.4295593,
    "startY": 36.1371692,
    "endX": 128.4328746,
    "endY": 36.13784296
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
print(response.json())  # JSON 응답 출력

200
{'usedFavoriteRouteVertices': '46235433,13008282:522,111:415,87:510,106:-257,-54', 'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [128.4295504193625, 36.13709548194513]}, 'properties': {'totalDistance': 440, 'totalTime': 83, 'totalFare': 0, 'taxiFare': 4000, 'index': 0, 'pointIndex': 0, 'name': '', 'description': '산호대로 을 따라  방면으로 307m 이동', 'nextRoadName': '산호대로', 'turnType': 200, 'pointType': 'S'}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[128.4295504193625, 36.13709548194513], [128.43100028240247, 36.13740380694986], [128.43204462826813, 36.13762324569759], [128.43215295139166, 36.1376454673903], [128.43285288539667, 36.13778990831635]]}, 'properties': {'index': 1, 'lineIndex': 0, 'name': '산호대로', 'description': '산호대로, 307m', 'distance': 307, 'time': 52, 'roadType': 5, 'facilityType': 0}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [128.43285288539667, 36.13778990831635]

In [51]:
# 받은 JSON 데이터 파싱
data = json.loads(response.text)
data['features']

[{'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [128.4295504193625, 36.13709548194513]},
  'properties': {'totalDistance': 440,
   'totalTime': 83,
   'totalFare': 0,
   'taxiFare': 4000,
   'index': 0,
   'pointIndex': 0,
   'name': '',
   'description': '산호대로 을 따라  방면으로 307m 이동',
   'nextRoadName': '산호대로',
   'turnType': 200,
   'pointType': 'S'}},
 {'type': 'Feature',
  'geometry': {'type': 'LineString',
   'coordinates': [[128.4295504193625, 36.13709548194513],
    [128.43100028240247, 36.13740380694986],
    [128.43204462826813, 36.13762324569759],
    [128.43215295139166, 36.1376454673903],
    [128.43285288539667, 36.13778990831635]]},
  'properties': {'index': 1,
   'lineIndex': 0,
   'name': '산호대로',
   'description': '산호대로, 307m',
   'distance': 307,
   'time': 52,
   'roadType': 5,
   'facilityType': 0}},
 {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [128.43285288539667, 36.13778990831635]},
  'properties': {'index': 2,
   'poin

In [50]:
distance_df = pd.DataFrame(data['features'][0].get('properties'), index=[0])
distance_df

,totalDistance,totalTime,totalFare,taxiFare,index,pointIndex,name,description,nextRoadName,turnType,pointType
0,440,83,0,4000,0,0,,산호대로 을 따라 방면으로 307m 이동,산호대로,200,S
